In [ ]:
# | default_exp nets/detr_3d

# Imports

In [ ]:
# | export


from functools import wraps
from typing import Literal

import torch
from einops import rearrange, repeat
from huggingface_hub import PyTorchModelHubMixin
from scipy.optimize import linear_sum_assignment
from torch import nn
from torch.nn import functional as F

from vision_architectures.blocks.transformer import Attention1DWithMLPConfig, TransformerDecoderBlock1D
from vision_architectures.docstrings import populate_docstring
from vision_architectures.layers.embeddings import AbsolutePositionEmbeddings3D, AbsolutePositionEmbeddings3DConfig
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import CustomBaseModel, Field, model_validator
from vision_architectures.utils.rearrange import rearrange_channels

# Config

In [ ]:
# | export


class DETRDecoderConfig(Attention1DWithMLPConfig):
    num_layers: int = Field(..., description="Number of transformer decoder layers.")


class DETRBBoxMLPConfig(CustomBaseModel):
    dim: int = Field(..., description="Dimension of the input features.")
    num_classes: int = Field(..., description="Number of classes for the bounding box predictions.")
    bbox_size_activation: Literal["sigmoid", "softplus"] = "sigmoid"

    @model_validator(mode="after")
    def validate(self):
        super().validate()
        assert self.num_classes > 0, "Number of classes must be greater than zero. If not predicting classes, set to 1."
        return self


class DETR3DConfig(DETRDecoderConfig, DETRBBoxMLPConfig, AbsolutePositionEmbeddings3DConfig):
    num_objects: int = Field(..., description="Maximum number of objects to detect.")
    drop_prob: float = Field(0.0, description="Dropout probability for input embeddings.")

# Architecture

### Decoder

In [ ]:
# | export


class DETRDecoder(nn.Module, PyTorchModelHubMixin):
    """DETR Transformer decoder."""

    @populate_docstring
    def __init__(self, config: DETRDecoderConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initialize the DETRDecoder. Activation checkpointing level 4.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__()

        self.config = DETRDecoderConfig.model_validate(config | kwargs)

        self.layers = nn.ModuleList(
            [TransformerDecoderBlock1D(config, checkpointing_level) for _ in range(self.config.num_layers)]
        )

        self.checkpointing_level4 = ActivationCheckpointing(4, checkpointing_level)

    @populate_docstring
    def _forward(
        self, object_queries: torch.Tensor, embeddings: torch.Tensor, return_intermediates: bool = False
    ) -> torch.Tensor | tuple[torch.Tensor, list[torch.Tensor]]:
        """Forward pass of the DETR3D decoder.

        Args:
            object_queries: Tokens that represent object queries. {INPUT_1D_DOC}
            embeddings: Actual embeddings of the input. {INPUT_1D_DOC}
            return_intermediates: If True, also returns the outputs of all layers. Defaults to False.

        Returns:
            If return_intermediates is True, returns the final object embeddings and a list of outputs from all layers.
            Otherwise, returns only the final object embeddings.
        """
        # object_queries: (b, num_possible_objects, dim)
        # embeddings: (b, num_embed_tokens, dim)

        object_embeddings = object_queries

        layer_outputs = []
        for layer in self.layers:
            object_embeddings = layer(object_embeddings, embeddings)
            layer_outputs.append(object_embeddings)

        if return_intermediates:
            return object_embeddings, layer_outputs
        return object_embeddings

    @wraps(_forward)
    def forward(self, *args, **kwargs):
        return self.checkpointing_level4(self._forward, *args, **kwargs)

In [ ]:
test_config = {
    "attn_drop_prob": 0.2,
    "dim": 54,
    "drop_prob": 0.2,
    "embed_spacing_info": False,
    "in_channels": 1,
    "mlp_ratio": 2,
    "layer_norm_eps": 1e-6,
    "learnable_absolute_position_embeddings": False,
    "mlp_drop_prob": 0.2,
    "num_heads": 6,
    "patch_size": (8, 16, 16),
    "proj_drop_prob": 0.2,
    "num_layers": 4,
}

test = DETRDecoder(test_config)
display(test)
o = test(
    torch.randn(2, 10, 54),
    torch.randn(2, 64, 54),
    True,
)
display((o[0].shape, [x.shape for x in o[1]]))


DETRDecoder(
  (layers): ModuleList(
    (0-3): 4 x TransformerDecoderBlock1D(
      (attn1): Attention1D(
        (W_q): Linear(in_features=54, out_features=54, bias=True)
        (W_k): Linear(in_features=54, out_features=54, bias=True)
        (W_v): Linear(in_features=54, out_features=54, bias=True)
        (proj): Linear(in_features=54, out_features=54, bias=True)
        (proj_drop): Dropout(p=0.2, inplace=False)
        (checkpointing_level1): ActivationCheckpointing(enabled=False)
        (checkpointing_level2): ActivationCheckpointing(enabled=False)
      )
      (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
      (attn2): Attention1D(
        (W_q): Linear(in_features=54, out_features=54, bias=True)
        (W_k): Linear(in_features=54, out_features=54, bias=True)
        (W_v): Linear(in_features=54, out_features=54, bias=True)
        (proj): Linear(in_features=54, out_features=54, bias=True)
        (proj_drop): Dropout(p=0.2, inplace=False)
        (


(
    torch.Size([2, 10, 54]),
    [
        torch.Size([2, 10, 54]),
        torch.Size([2, 10, 54]),
        torch.Size([2, 10, 54]),
        torch.Size([2, 10, 54])
    ]
)

In [ ]:
# | export


class DETRBBoxMLP(nn.Module):
    """DETR Bounding Box MLP. This module predicts bounding boxes and class scores from object query embeddings."""

    @populate_docstring
    def __init__(self, config: DETRBBoxMLPConfig = {}, **kwargs):
        """Initialize the DETRBBoxMLP.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__()

        self.config = DETRBBoxMLPConfig.model_validate(config | kwargs)

        self.linear = nn.Linear(self.config.dim, 6 + 1 + self.config.num_classes)

    @populate_docstring
    def forward(
        self,
        object_embeddings: torch.Tensor,
    ) -> torch.Tensor:
        """Forward pass of the DETRBBoxMLP.

        Args:
            object_embeddings: Object embeddings from the DETR decoder. {INPUT_1D_DOC}

        Returns:
            A tensor of shape (b, num_possible_objects, 1 objectness class + 6 bounding box parameters + num_classes)
            containing the predicted bounding boxes and class scores. {BOUNDING_BOXES_FORMAT_DOC}
        """
        # object_embeddings: (b, num_possible_objects, dim)

        bboxes = self.linear(object_embeddings)
        # (b, num_possible_objects, 6 + 1 + num_classes)

        # Bound the bounding box parameters
        centers, sizes, logits = bboxes[:, :, :3], bboxes[:, :, 3:6], bboxes[:, :, 6:]
        centers = centers.sigmoid()  # center coordinates should be in the bbox
        if self.config.bbox_size_activation == "sigmoid":
            sizes = sizes.sigmoid()  # sizes should be between 0 and 1
        elif self.config.bbox_size_activation == "softplus":
            sizes = F.softplus(sizes)  # sizes should be positive but unbounded
        bboxes = torch.cat([centers, sizes, logits], dim=-1)

        return bboxes

In [ ]:
test_config = {
    "dim": 54,
    "num_classes": 10,
}

test = DETRBBoxMLP(test_config)
display(test)
o = test(
    torch.randn(2, 10, 54),
)
display((o[0].shape), o[0][0])


DETRBBoxMLP(
  (linear): Linear(in_features=54, out_features=17, bias=True)
)

torch.Size([10, 17])


tensor([ 0.5408,  0.4994,  0.4254,  0.5085,  0.3362,  0.4264, -1.2220, -0.0613,
         1.0281, -0.3249,  0.3751, -0.9232,  1.4117,  0.1219,  1.3651,  0.2143,
         0.3949], grad_fn=<SelectBackward0>)

# Models

In [ ]:
# | export


class DETR3D(nn.Module, PyTorchModelHubMixin):
    """DETR 3D model. Also implements bipartite matching loss which is essential for DETR training."""

    @populate_docstring
    def __init__(self, config: DETR3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initialize the DETR3D. Activation checkpointing level 4.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__()

        self.config = DETR3DConfig.model_validate(config | kwargs)

        self.position_embeddings = AbsolutePositionEmbeddings3D(config)
        self.pos_drop = nn.Dropout(self.config.drop_prob)
        self.num_possible_objects = self.config.num_objects
        self.object_queries = nn.Parameter(torch.randn(1, self.num_possible_objects, self.config.dim) * 0.01)
        self.decoder = DETRDecoder(config, checkpointing_level)
        self.bbox_mlp = DETRBBoxMLP(config)

        self.class_prevalences = [None] * (self.config.num_classes + 1)  # used to weight cross entropy loss

        self.checkpointing_level4 = ActivationCheckpointing(4, checkpointing_level)

    @populate_docstring
    def _forward(
        self,
        embeddings: torch.Tensor,
        spacings: torch.Tensor | None = None,
        channels_first: bool = True,
        return_intermediates: bool = False,
    ) -> torch.Tensor | tuple[torch.Tensor, torch.Tensor, list[torch.Tensor]]:
        """Forward pass of the DETR3D.

        Args:
            embeddings: Encoded input features. {INPUT_3D_DOC}
            spacings: {SPACINGS_DOC}
            channels_first: {CHANNELS_FIRST_DOC}
            return_intermediates: If True, also returns the outputs of all layers. Defaults to False.

        Returns:
            A tuple containing bounding boxes, object embeddings, and layer outputs if return_intermediates is True.
            Else, returns only the bounding boxes. {BOUNDING_BOXES_FORMAT_DOC}
        """
        # embeddings: (b, [dim], num_tokens_z, num_tokens_y, num_tokens_x, [dim])
        # spacings: (b, 3)

        embeddings = rearrange_channels(embeddings, channels_first, True)
        # (b, dim, num_tokens_z, num_tokens_y, num_tokens_x)

        embeddings = self.position_embeddings(embeddings, spacings=spacings)
        embeddings = self.pos_drop(embeddings)
        # (b, dim, num_tokens_z, num_tokens_y, num_tokens_x)

        embeddings = rearrange(embeddings, "b d z y x -> b (z y x) d")
        # (b, num_embed_tokens, dim)

        object_queries = repeat(self.object_queries, "1 n d -> b n d", b=embeddings.shape[0])
        # (b, num_possible_objects, dim)

        object_embeddings, layer_outputs = self.decoder(object_queries, embeddings, return_intermediates=True)
        # object_embeddings: (b, num_possible_objects, dim)
        # layer_outputs: list of (b, num_possible_objects, dim)

        bboxes = self.bbox_mlp(object_embeddings)
        # (b, num_possible_objects, 6 + 1 + num_classes)

        if return_intermediates:
            return bboxes, object_embeddings, layer_outputs

        return bboxes

    @wraps(_forward)
    def forward(self, *args, **kwargs):
        return self.checkpointing_level4(self._forward, *args, **kwargs)

    def bipartite_matching_loss(
        self,
        pred: torch.Tensor,
        target: torch.Tensor | list[torch.Tensor],
        classification_cost_weight: float = 1.0,
        bbox_l1_cost_weight: float = 1.0,
        bbox_iou_cost_weight: float = 1.0,
        weight_loss_based_on_prevalence: bool = True,
        reduction: str = "mean",
        return_matching: bool = False,
        return_loss_components: bool = False,
    ) -> torch.Tensor | tuple[torch.Tensor, list] | tuple[torch.Tensor, dict] | tuple[torch.Tensor, list, dict]:
        """Bipartite matching loss for DETR. The classes are expected to optimize for a multi-class classification
        problem. Expects raw logits in class predictions, not probabilities. Use ``logits_to_scores_fn=None`` in the
        ``forward`` function to avoid applying any transformation.

        Args:
            pred: Predicted bounding boxes and class scores. It should be of shape
                `(B, num_objects, 6 + 1 + num_classes)`. Number of objects and number of classes will be inferred from
                here.
            target: Target bounding boxes and class scores. If provided as a list, each element should be a tensor for
                the corresponding batch element in ``pred`` and therefore should have a length of `B`. Each tensor
                should have less than or equal to the number of objects in `pred`. The number of classes can either be
                the exact same as in `pred`, or it should be 1 argmax (one-cold) decoding.
            classification_cost_weight: Weight for the classification cost in hungarian matching.
            bbox_l1_cost_weight: Weight for the bounding box L1 loss cost in hungarian matching.
            bbox_iou_cost_weight: Weight for the bounding box IoU cost in hungarian matching.
            weight_loss_based_on_prevalence: Whether to weight the classification loss based on the prevalence of
                each class in the dataset. If True, the loss will be weighted by the inverse prevalence of each class.
            reduction: Specifies the reduction to apply to the output.
            return_matching: Whether or not to return the matched indices from the bipartite matching.
            return_loss_components: Whether or not to return the individual loss components.

        Returns:
            A tensor containing the bipartite matching loss with the shape depending on the `reduction` argument. If
            `return_matching` is True, also returns a list of tuples containing matched indices for predictions and
            targets. Each tuple is of the form `(pred_indices, target_indices)`, where `pred_indices` and
            `target_indices` are lists of indices for the matched predictions and targets, respectively. If
            `return_loss_components` is True, also returns a dict of each loss component reduced based on `reduction`.
        """
        B = pred.shape[0]

        # Convert target to a list of tensors if not already
        if isinstance(target, torch.Tensor):
            target = list(target)

        # argmax encode the class labels if they are not already
        for i in range(len(target)):
            if target[i].shape[-1] > 7:  # 6 bbox + 1 class
                target[i] = torch.cat([target[i][:, :6], target[i][:, 6:].argmax(-1, keepdims=True)], dim=-1)

        # Perform hungarian matching
        matched_indices = DETR3D.hungarian_matching(
            pred, target, classification_cost_weight, bbox_l1_cost_weight, bbox_iou_cost_weight
        )

        # Update class prevalences and calculate cross entropy loss weights if applicable
        self._update_class_prevalences(target)
        if weight_loss_based_on_prevalence:
            classification_loss_weights = self._get_cross_entropy_loss_weights(pred.device)
        else:
            classification_loss_weights = torch.ones(
                (self.config.num_classes + 1,), dtype=torch.float32, device=pred.device
            )

        losses = {
            "classification_loss": [],
            "bbox_l1_loss": [],
            "bbox_iou_loss": [],
            "total_loss": [],
        }
        for i in range(B):
            batch_losses = {}

            pred_indices, target_indices = matched_indices[i]

            # Classification loss for ALL predictions
            all_class_labels = torch.zeros_like(pred[i][:, 6], dtype=torch.long)
            all_class_labels[pred_indices] = target[i][target_indices, 6].long()
            all_pred_classes = pred[i][..., 6:]
            batch_losses["classification_loss"] = F.cross_entropy(
                all_pred_classes, all_class_labels, weight=classification_loss_weights
            )

            # Calculate all other losses only if prediction and target have objects
            if pred_indices != [] and target_indices != []:
                matched_pred = pred[i][pred_indices]
                matched_target = target[i][target_indices]

                pred_bboxes = matched_pred[:, :6].clone()
                target_bboxes = matched_target[:, :6].clone()

                # Compute losses for matched pairs
                # BBox L1 loss
                batch_losses["bbox_l1_loss"] = F.l1_loss(pred_bboxes, target_bboxes)

                # For IOU calculation, it does not matter if bboxes are in actual pixel lengths or normalized based on
                # image size, metric value will be the same.

                # BBox IOU loss
                batch_losses["bbox_iou_loss"] = 1 - DETR3D._generalized_bbox_iou(pred_bboxes, target_bboxes).mean()

            # Total loss for this batch element
            total_loss = (
                classification_cost_weight * batch_losses.get("classification_loss", 0.0)
                + bbox_l1_cost_weight * batch_losses.get("bbox_l1_loss", 0.0)
                + bbox_iou_cost_weight * batch_losses.get("bbox_iou_loss", 0.0)
            )

            # Add to losses
            losses["classification_loss"].append(
                batch_losses.get("classification_loss", torch.tensor(torch.nan, device=pred.device)),
            )
            losses["bbox_l1_loss"].append(
                batch_losses.get("bbox_l1_loss", torch.tensor(torch.nan, device=pred.device)),
            )
            losses["bbox_iou_loss"].append(
                batch_losses.get("bbox_iou_loss", torch.tensor(torch.nan, device=pred.device)),
            )
            losses["total_loss"].append(total_loss)

        # Stack batch losses and apply reduction
        for key in losses:
            loss = torch.stack(losses[key])

            if reduction == "mean":
                loss = loss.nanmean()
            elif reduction == "sum":
                loss = loss.nansum()
            elif reduction == "none" or reduction is None:
                pass
            else:
                raise ValueError(f"Invalid reduction mode: {reduction}")

            losses[key] = loss

        loss = losses["total_loss"]
        losses.pop("total_loss", None)

        match return_matching, return_loss_components:
            case False, False:
                return loss
            case True, False:
                return loss, matched_indices
            case False, True:
                return loss, losses
            case True, True:
                return loss, matched_indices, losses

    @torch.no_grad()
    @staticmethod
    def hungarian_matching(
        pred: torch.Tensor,
        target: list[torch.Tensor],
        classification_cost_weight: float = 1.0,
        bbox_l1_cost_weight: float = 1.0,
        bbox_iou_cost_weight: float = 1.0,
    ) -> list[tuple[list[int], list[int]]]:
        """Hungarian matching between predictions and targets.

        Args:
            pred: Predicted bounding boxes and class scores. It should be of shape
                `(B, num_objects, 6 + 1 + num_classes)`. Number of objects and number of classes will be inferred from
                here.
            target: Target bounding boxes and class scores. This is expected in argmax encoding or one-hot encoding.
            classification_cost_weight: Weight for the classification cost.
            bbox_l1_cost_weight: Weight for the bounding box L1 loss cost.
            bbox_iou_cost_weight: Weight for the bounding box IoU cost.

        Returns:
            A list of tuples containing matched indices for predictions and targets. Each tuple is of the form
            `(pred_indices, target_indices)`, where `pred_indices` and `target_indices` are lists of indices for the
            matched predictions and targets, respectively.
        """
        B = pred.shape[0]

        matched_indices = []
        for i in range(B):
            pred_bboxes = pred[i, :, :6]  # (num_objects, 6)
            target_bboxes = target[i][:, :6]  # (<=num_objects, 6)

            # If either prediction or target has no objects, skip matching
            if pred_bboxes.shape[0] == 0 or target_bboxes.shape[0] == 0:
                matched_indices.append(([], []))
                continue

            pred_class_logits = pred[i, :, 6:]  # (num_objects, num_classes)
            target_class_labels = target[i][:, 6].long()  # (<=num_objects,) this is in argmax encoding

            # ----- Cost matrix calculation -----

            # Classification cost
            pred_class_probabilities = F.softmax(pred_class_logits, dim=-1)
            # (num_objects, num_classes)

            classification_cost = -pred_class_probabilities[:, target_class_labels]
            # (num_objects, <=num_objects)

            # L1 loss for bounding boxes
            bbox_l1_cost = torch.cdist(pred_bboxes, target_bboxes, p=1)
            # (num_objects, <=num_objects)

            # IOU cost for bounding boxes
            bbox_iou_cost = 1 - DETR3D._generalized_pairwise_bbox_iou(pred_bboxes, target_bboxes)
            # (num_objects, <=num_objects)

            # Total cost matrix
            cost_matrix = (
                classification_cost_weight * classification_cost
                + bbox_l1_cost_weight * bbox_l1_cost
                + bbox_iou_cost_weight * bbox_iou_cost
            )
            # (num_objects, <=num_objects)

            # Hungarian matching
            pred_indices_element, target_indices_element = linear_sum_assignment(cost_matrix.detach().cpu().numpy())

            matched_indices.append((list(pred_indices_element), list(target_indices_element)))

        return matched_indices

    @staticmethod
    def _generalized_bbox_iou(
        pred_bboxes: torch.Tensor,
        target_bboxes: torch.Tensor,
    ) -> torch.Tensor:
        """Compute the IoU loss between two matched sets of bounding boxes.

        Args:
            pred_bboxes: Predicted bounding boxes of shape `(num_boxes, 6)`. {BOUNDING_BOXES_FORMAT_DOC}
            target_bboxes: Target bounding boxes of shape `(num_boxes, 6)`. {BOUNDING_BOXES_FORMAT_DOC}

        Returns:
            A tensor containing the IoU loss.
        """

        # Convert bboxes from center format (z, y, x, d, h, w) to corner format
        def center_to_corners(bboxes):
            centers = bboxes[:, :3]
            sizes = bboxes[:, 3:] / 2
            min_coords = centers - sizes
            max_coords = centers + sizes
            return torch.cat([min_coords, max_coords], dim=1)  # shape (N, 6)

        pred_corners = center_to_corners(pred_bboxes)
        target_corners = center_to_corners(target_bboxes)

        # Intersection corners
        max_min = torch.max(pred_corners[:, :3], target_corners[:, :3])
        min_max = torch.min(pred_corners[:, 3:], target_corners[:, 3:])
        inter_dims = (min_max - max_min).clamp(min=0)
        inter_vol = inter_dims.prod(dim=1)

        # Volumes
        pred_dims = pred_corners[:, 3:] - pred_corners[:, :3]
        target_dims = target_corners[:, 3:] - target_corners[:, :3]
        pred_vol = pred_dims.prod(dim=1)
        target_vol = target_dims.prod(dim=1)
        union_vol = pred_vol + target_vol - inter_vol

        # Enclosing box corners
        enc_min = torch.min(pred_corners[:, :3], target_corners[:, :3])
        enc_max = torch.max(pred_corners[:, 3:], target_corners[:, 3:])
        enc_dims = (enc_max - enc_min).clamp(min=0)
        enc_vol = enc_dims.prod(dim=1)

        iou = inter_vol / union_vol.clamp(min=1e-7)
        gious = iou - (enc_vol - union_vol) / enc_vol.clamp(min=1e-7)

        return gious

    @staticmethod
    def _generalized_pairwise_bbox_iou(
        pred_bboxes: torch.Tensor,
        target_bboxes: torch.Tensor,
    ) -> torch.Tensor:
        """Compute the IoU loss between all combinations of predicted and target bounding boxes.

        Args:
            pred_bboxes: Predicted bounding boxes of shape `(num_objects, 6)`. {BOUNDING_BOXES_FORMAT_DOC}
            target_bboxes: Target bounding boxes of shape `(<=num_objects, 6)`. {BOUNDING_BOXES_FORMAT_DOC}

        Returns:
            A tensor containing the IoU losses of all combinations.
        """
        # Compute pairwise IoU
        gious = []
        for i in range(pred_bboxes.shape[0]):
            row_ious = []
            for j in range(target_bboxes.shape[0]):
                row_ious.append(DETR3D._generalized_bbox_iou(pred_bboxes[i : i + 1], target_bboxes[j : j + 1]).mean())

            gious.append(torch.stack(row_ious))

        return torch.stack(gious)

    def _update_class_prevalences(self, target: list[torch.Tensor]):
        """Update the class prevalences based on the classes present in the ground truth

        Args:
            target: A list of ground truth tensors, each of shape (num_objects, 7) where the last dimension contains the
                bounding box class.
        """
        # Store the number of times each class has been encountered in this batch
        class_counts = [0] * (self.config.num_classes + 1)
        B = len(target)
        for b in range(B):
            # If there are more than self.config.num_objects targets, only the first self.config.num_objects will be
            # considered
            for c in target[b][: self.config.num_objects, -1].int():
                class_counts[c] += 1

            # For background class, prevalence is the number of extra bboxes that were predicted
            class_counts[0] += max(0, self.config.num_objects - len(target[b]))
            # max in case there are more target bboxes than self.config.num_objects

        # Calculate prevalence of each class
        new_prevalences = [count / sum(class_counts) for count in class_counts]

        # Update current prevalences using EMA to allow for distribution shift in data
        for i in range(self.config.num_classes + 1):
            if self.class_prevalences[i] is None:
                if new_prevalences[i] > 0:
                    self.class_prevalences[i] = new_prevalences[i]
                # otherwise let it stay None
            else:
                # update to new value using EMA
                self.class_prevalences[i] = self.class_prevalences[i] * 0.99 + new_prevalences[i] * 0.01

    def _get_cross_entropy_loss_weights(self, device=torch.device("cpu")) -> torch.Tensor:
        """Get the weights for the cross entropy loss based on class prevalences

        Returns:
            A tensor containing the weights for the cross entropy loss of shape (num_classes + 1,).
        """
        # Get class prevalences and replace None with nan
        class_prevalences = [
            prevalence if prevalence is not None else torch.nan for prevalence in self.class_prevalences
        ]

        # Calculate weights as inverse of class prevalences
        weights = 1 / torch.tensor(class_prevalences, dtype=torch.float32, device=device)

        # Substitute nan values with mean and clamp weights to a limit
        # Assumption: all classes are visited at least once in the dataset
        mu, std = weights.nanmean(), weights[~weights.isnan()].std()
        weights = weights.nan_to_num(mu)
        weights = weights.clamp(mu - 3 * std, mu + 3 * std)

        # Normalize weights to sum to (self.config.num_classes + 1)
        weights = (self.config.num_classes + 1) * weights / weights.sum()

        return weights

In [ ]:
test_config = {
    "patch_size": (8, 16, 16),
    "in_channels": 1,
    "dim": 54,
    "num_heads": 6,
    "mlp_ratio": 2,
    "layer_norm_eps": 1e-6,
    "attn_drop_prob": 0.2,
    "proj_drop_prob": 0.2,
    "mlp_drop_prob": 0.2,
    "learnable_absolute_position_embeddings": True,
    "embed_spacing_info": False,
    "image_size": (32, 512, 512),
    "num_objects": 10,
    "num_classes": 5,
    "num_layers": 4,
}

test = DETR3D(test_config)
display(test)
o = test(
    torch.randn(2, 1, 4, 32, 32),
    torch.randn(2, 3),
    return_intermediates=True,
)
display((o[0].shape, o[1].shape, [x.shape for x in o[2]]))

for gt_bboxes in [
    [
        torch.cat([torch.rand(10, 6), torch.randint(0, 6, (10, 1))], dim=-1),
        torch.cat([torch.rand(2, 6), torch.randint(0, 6, (2, 1))], dim=-1),
    ],  # Regular testing
    [
        torch.rand(10, 12),
        torch.rand(2, 12),
    ],  # Requiring argmax encoding
    [
        torch.tensor([]).reshape(0, 12),
        torch.tensor([]).reshape(0, 7),
    ],  # No ground truth boxes, classification loss should still be calculated
]:
    display(test.bipartite_matching_loss(o[0], gt_bboxes, reduction="none", return_loss_components=True))


DETR3D(
  (position_embeddings): AbsolutePositionEmbeddings3D()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (decoder): DETRDecoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerDecoderBlock1D(
        (attn1): Attention1D(
          (W_q): Linear(in_features=54, out_features=54, bias=True)
          (W_k): Linear(in_features=54, out_features=54, bias=True)
          (W_v): Linear(in_features=54, out_features=54, bias=True)
          (proj): Linear(in_features=54, out_features=54, bias=True)
          (proj_drop): Dropout(p=0.2, inplace=False)
          (checkpointing_level1): ActivationCheckpointing(enabled=False)
          (checkpointing_level2): ActivationCheckpointing(enabled=False)
        )
        (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
        (attn2): Attention1D(
          (W_q): Linear(in_features=54, out_features=54, bias=True)
          (W_k): Linear(in_features=54, out_features=54, bias=True)
          (W_v): Linear(in_features=54, ou


(
    torch.Size([2, 10, 12]),
    torch.Size([2, 10, 54]),
    [
        torch.Size([2, 10, 54]),
        torch.Size([2, 10, 54]),
        torch.Size([2, 10, 54]),
        torch.Size([2, 10, 54])
    ]
)


(
    tensor([4.3433, 3.2983], grad_fn=<StackBackward0>),
    {
        'classification_loss': tensor([2.5759, 1.7373], grad_fn=<StackBackward0>),
        'bbox_l1_loss': tensor([0.2747, 0.1793], grad_fn=<StackBackward0>),
        'bbox_iou_loss': tensor([1.4927, 1.3817], grad_fn=<StackBackward0>)
    }
)


(
    tensor([3.7995, 4.0535], grad_fn=<StackBackward0>),
    {
        'classification_loss': tensor([2.0776, 2.3716], grad_fn=<StackBackward0>),
        'bbox_l1_loss': tensor([0.2339, 0.2273], grad_fn=<StackBackward0>),
        'bbox_iou_loss': tensor([1.4880, 1.4546], grad_fn=<StackBackward0>)
    }
)


(
    tensor([0.9979, 1.1660], grad_fn=<StackBackward0>),
    {
        'classification_loss': tensor([0.9979, 1.1660], grad_fn=<StackBackward0>),
        'bbox_l1_loss': tensor([nan, nan]),
        'bbox_iou_loss': tensor([nan, nan])
    }
)

# nbdev

In [ ]:
!nbdev_export